In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 4, Finished, Available, Finished)

In [3]:
Returns_df = spark.read.format('csv')\
                .option('header',True)\
                .option('inferschema',True)\
                .load('abfss://Fabric_Demo@onelake.dfs.fabric.microsoft.com/Lakehouse_Demo.Lakehouse/Files/Sales_Bronze/Raw_Data/Returns.csv')


StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 5, Finished, Available, Finished)

In [4]:
display(Returns_df.head(5))

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3720825-87c3-450c-b06a-f8dd6ef26b8c)

In [5]:
Sales_df = spark.read.format('csv')\
                  .option('header',True)\
                  .option('inferschema',True)\
                  .load('abfss://Fabric_Demo@onelake.dfs.fabric.microsoft.com/Lakehouse_Demo.Lakehouse/Files/Sales_Bronze/Raw_Data/Sales.csv')

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 7, Finished, Available, Finished)

In [6]:
display(Sales_df.head(5))

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 84fd78ed-b616-4cc4-80e7-85153f83cb03)

In [73]:
df_final = Sales_df.join(Returns_df, Sales_df.Order_ID == Returns_df.Order_ID, how = 'left')\
                   .drop(Returns_df.Order_ID, Returns_df.Customer_Name, Returns_df.Sales_Amount)

display(df_final.head(5))

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 75, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d361e4f2-dd97-430e-beaa-5c7ad3d240eb)

In [74]:
print(df_final.columns)


StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 76, Finished, Available, Finished)

['Order_ID', 'Order_Date', 'Shipping_Date', 'Aging', 'Ship_Mode', 'Product_Category', 'Product', ' Sales ', 'Quantity', 'Discount', ' Profit ', ' Shipping_Cost ', 'Order_Priority', 'Customer_ID', 'Customer_Name', 'Segment', 'City', 'State', 'Country', 'Region', 'Return']


In [76]:
df_final = df_final.toDF(*[c.strip() for c in df_final.columns])
print(df_final.columns)


StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 78, Finished, Available, Finished)

['Order_ID', 'Order_Date', 'Shipping_Date', 'Aging', 'Ship_Mode', 'Product_Category', 'Product', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping_Cost', 'Order_Priority', 'Customer_ID', 'Customer_Name', 'Segment', 'City', 'State', 'Country', 'Region', 'Return']


In [87]:
df_final = df_final.withColumn("Order_Date",to_date(col("Order_Date"), "M/d/yyyy"))

df_final = df_final.withColumn("Order_Date", to_date(col("Order_Date"), "yyyy-MM-dd"))

df_final = df_final.withColumn("Shipping_Date",to_date(col("Shipping_Date"), "M/d/yyyy"))

df_final = df_final.withColumn("Shipping_Date", to_date(col("Shipping_Date"), "yyyy-MM-dd"))

df_final = df_final.withColumn("Aging", col("Aging").cast("int"))

df_final = df_final.withColumn("Quantity", col("Quantity").cast("int"))

df_final = df_final.withColumn("Sales",regexp_replace(col("Sales"), "[$]", "").cast("double"))

df_final = df_final.withColumn("Discount", col("Discount").cast("double"))

df_final = df_final.withColumn("Profit",regexp_replace(col("Profit"), "[$]", "").cast("double"))

df_final = df_final.withColumn("Shipping_Cost",regexp_replace(col("Shipping_Cost"), "[$]", "").cast("double"))

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 89, Finished, Available, Finished)

In [88]:
df_mod = df_final.withColumns({"Order_Year": year("Order_Date"), \
                "Order_Month": month("Order_Date"), \
                "Created_TS": current_timestamp(),\
                "Modified_TS": current_timestamp(),\
                })

display(df_mod.head(5))

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 90, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 29ea7943-3c0d-4329-90ac-971aecee983a)

In [89]:
df_mod.createOrReplaceTempView("ViewSales")

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 91, Finished, Available, Finished)

In [90]:
%%sql
SELECT * from ViewSales LIMIT 5

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 92, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 25 fields>

In [85]:
%%sql
create table if not exists Lakehouse_Demo.Sales_Stg
(
    Order_ID	string,
    Order_Date	Date,
    Shipping_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	double,
    Quantity	int,
    Discount	 double,
    Profit 	 double,
    Shipping_Cost 	double,
    Order_Priority	string,
    Customer_ID	string,
    Customer_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
using DELTA
PARTITIONED by(Order_Year,Order_Month)


StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 87, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [84]:
%%sql
drop table Lakehouse_Demo.Sales_Stg

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 86, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [92]:
%%sql
MERGE INTO Lakehouse_Demo.Sales_Stg AS SS
USING ViewSales AS VS
ON SS.Order_Year = VS.Order_Year
  AND SS.Order_Month = VS.Order_Month
  AND SS.Order_ID = VS.Order_ID

WHEN MATCHED THEN 
UPDATE SET
    SS.Order_Date        = VS.Order_Date,
    SS.Shipping_Date     = VS.Shipping_Date,
    SS.Aging             = VS.Aging,
    SS.Ship_Mode         = VS.Ship_Mode,
    SS.Product_Category  = VS.Product_Category,
    SS.Product           = VS.Product,
    SS.Sales             = VS.Sales,
    SS.Quantity          = VS.Quantity,
    SS.Discount          = VS.Discount,
    SS.Profit            = VS.Profit,
    SS.Shipping_Cost     = VS.Shipping_Cost,
    SS.Order_Priority    = VS.Order_Priority,
    SS.Customer_ID       = VS.Customer_ID,
    SS.Customer_Name     = VS.Customer_Name,
    SS.Segment           = VS.Segment,
    SS.City              = VS.City,
    SS.State             = VS.State,
    SS.Country           = VS.Country,
    SS.Region            = VS.Region,
    SS.Return            = VS.Return,
    SS.Modified_TS       = VS.Modified_TS

WHEN NOT MATCHED THEN 
INSERT (
    Order_ID,
    Order_Date,
    Shipping_Date,
    Aging,
    Ship_Mode,
    Product_Category,
    Product,
    Sales,
    Quantity,
    Discount,
    Profit,
    Shipping_Cost,
    Order_Priority,
    Customer_ID,
    Customer_Name,
    Segment,
    City,
    State,
    Country,
    Region,
    Return,
    Order_Year,
    Order_Month,
    Created_TS,
    Modified_TS
)
VALUES (
    VS.Order_ID,
    VS.Order_Date,
    VS.Shipping_Date,
    VS.Aging,
    VS.Ship_Mode,
    VS.Product_Category,
    VS.Product,
    VS.Sales,
    VS.Quantity,
    VS.Discount,
    VS.Profit,
    VS.Shipping_Cost,
    VS.Order_Priority,
    VS.Customer_ID,
    VS.Customer_Name,
    VS.Segment,
    VS.City,
    VS.State,
    VS.Country,
    VS.Region,
    VS.Return,
    VS.Order_Year,
    VS.Order_Month,
    VS.Created_TS,
    VS.Modified_TS
);


StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 94, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [93]:
%%sql
Select * from Lakehouse_Demo.Sales_Stg

StatementMeta(, 633f55b3-47cb-468f-87de-4a5293576126, 95, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 25 fields>